In [4]:
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, AutoProcessor
from PIL import Image
from torch.utils.data import Dataset
import torch
import requests
import json
import os
import numpy as np

In [21]:

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-base")

processor = AutoProcessor.from_pretrained("google/pix2struct-base")

image = Image.open('/home/ges/level3-cv-productserving-cv-10/data/infographics/images/36966.jpeg')
text = "What was the name of 'Snapchat' when it was launched?"

inputs = processor(images=image, return_tensors="pt")
labels = processor(text=text, return_tensors="pt").input_ids

print(labels)
# forward pass
outputs = model(**inputs, labels=labels)


loss = outputs.loss
print(f"{loss.item():.5f}")

tensor([[  484,   306,   276,   560,   280,   273,   284,   473, 46511, 16442,
           284,   364,   298,   306,  4684,   311,     1]])
3.92802


7.22370


In [26]:
from transformers import Pix2StructProcessor
from PIL import Image
from torch.utils.data import Dataset

class InfographicsDataset(Dataset):
    def __init__(self, img_paths, questions, answers, processor):
        self.img_paths = img_paths
        self.questions = questions
        self.answers = answers
        self.processor = processor
        
    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        
        question = self.questions[idx]
        answer = self.answers[idx]
        
        # 이미지와 질문을 함께 처리
        inputs = self.processor(text=question, images=image, return_tensors="pt", padding=True, truncation=True)
        
        # 정답(라벨)을 모델이 처리할 수 있는 형식으로 변환 (이 부분은 모델과 태스크에 따라 달라집니다)
        # 예제에서는 단순화를 위해 answers가 이미 적절한 형식이라고 가정
        # # labels = self.processor(text=answer, return_tensors="pt").input_ids
        # answers_tokens = self.processor(text=answer, return_tensors="pt", padding=True, truncation=True).input_ids
        # labels = answers_tokens
        
        return inputs, labels
        
        


In [27]:
import torch
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor, AdamW
from torch.utils.data import DataLoader

with open('/home/ges/level3-cv-productserving-cv-10/data/infographics/qas/infographicsVQA_train_v1.0.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

img_paths = []
questions = []
answers = []

for item in data['data']:
    img_paths.append('/home/ges/level3-cv-productserving-cv-10/data/infographics/images/'+ item['image_local_name'])
    questions.append(item['question'])
    answers.append(item['answers'])


dataset = InfographicsDataset(img_paths, questions, answers, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-infographics-vqa-large").to("cuda")
optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(10):  # num_epochs is a variable you should define
    for batch in dataloader:
        inputs, labels = batch
        # Adjust `inputs` and `labels` here if necessary to fit your task

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")

RuntimeError: stack expects each tensor to be equal size, but got [1, 21] at entry 0 and [1, 6] at entry 1

In [14]:


# Assuming `img_paths` and `annotations` are lists containing your data.



print(len(img_paths) == len(questions) == len(answers))

processor = Pix2StructProcessor.from_pretrained("google/pix2struct-base")
model = Pix2StructForConditionalGeneration.from_pretrained("google/pix2struct-base")

dataset = InfographicsDataset(img_paths, questions, answers, processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
num_epochs = 5
for epoch in range(num_epochs):  # num_epochs is a variable you should define
    for batch in dataloader:
        pass
        # inputs, labels = batch
        # # Adjust `inputs` and `labels` here if necessary to fit your task

        # outputs = model(**inputs, labels=labels)
        # loss = outputs.loss

        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()

        # print(f"Epoch {epoch}, Loss: {loss.item()}")


True


/home/ges/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'flattened_patches': tensor([[[ 1.0000,  1.0000, -2.1435,  ..., -0.2347, -1.6987, -1.7526],
         [ 1.0000,  2.0000, -1.4501,  ..., -0.2523, -1.7257, -1.6720],
         [ 1.0000,  3.0000, -1.4023,  ...,  0.4378, -0.0875, -0.7754],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]), 'attention_mask': tensor([[1., 1., 1.,  ..., 0., 0., 0.]]), 'decoder_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'decoder_input_ids': tensor([[1609,  313,  276,  754,  910, 2213,  299, 4162,  577, 8246,    1]])}
{'flattened_patches': tensor([[[ 1.0000,  1.0000, -2.5451,  ...,  0.8412,  0.8412,  0.8412],
         [ 1.0000,  2.0000, -2.5999,  ...,  0.8524,  0.8524,  0.8524],
         [ 1.0000,  3.0000, -2.5999,  ...,  0.8617,  0.8617,  0.8617],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0

RuntimeError: stack expects each tensor to be equal size, but got [1, 11] at entry 0 and [1, 15] at entry 1